In [8]:
# Analizzo tutte le colonne del dataset per identificare domande mancanti
# Prima carico i dati da uno dei file merged esistenti
import pandas as pd

# Proviamo prima con il file di backup o con uno dei file merged
file_paths = [
    './dati backup/cpia.xlsx',
    './webapp/backend/uploads/projects/4bed8f4c/merged_20250910_085926.xlsx',
    './webapp/backend/uploads/projects/34091dcf/merged_20250910_085429.xlsx',
    'merged_results.xlsx'
]

merged = None
for file_path in file_paths:
    try:
        print(f"Tentativo di caricamento: {file_path}")
        merged = pd.read_excel(file_path)
        if merged.shape[0] > 0 and merged.shape[1] > 0:
            print(f"File caricato correttamente! Shape: {merged.shape}")
            print(f"File utilizzato: {file_path}")
            break
        else:
            print(f"File vuoto: {merged.shape}")
    except Exception as e:
        print(f"Errore nel caricamento: {e}")

if merged is None or merged.shape[0] == 0:
    print("Impossibile caricare alcun file con dati!")
    exit()

print("=== ANALISI COMPLETA DELLE COLONNE NEL DATASET ===")
print(f"Numero totale di colonne: {len(merged.columns)}")
print()

# Estraggo tutti i numeri delle domande dalle colonne
import re
question_numbers = set()
question_columns = {}

pattern = r'^(\d+\.\d+(?:\.\d+)?)'
for col in merged.columns:
    match = re.match(pattern, col)
    if match:
        q_num = match.group(1)
        question_numbers.add(q_num)
        if q_num not in question_columns:
            question_columns[q_num] = []
        question_columns[q_num].append(col)

# Ordino i numeri delle domande
sorted_questions = sorted(question_numbers, key=lambda x: [int(n) for n in x.split('.')])

print(f"Domande trovate nel dataset ({len(sorted_questions)}):")
for i, q_num in enumerate(sorted_questions, 1):
    col_count = len(question_columns[q_num])
    print(f"{i:2d}. {q_num} ({col_count} colonne)")

print()
print("=== ANALISI NUMERAZIONE SEQUENZIALE ===")

# Controllo le sequenze per sezione
sections = {}
for q_num in sorted_questions:
    parts = q_num.split('.')
    if len(parts) >= 2:
        section = parts[0]
        if section not in sections:
            sections[section] = []
        sections[section].append(q_num)

for section in sorted(sections.keys(), key=int):
    section_questions = sorted(sections[section], key=lambda x: [int(n) for n in x.split('.')])
    print(f"\nSezione {section}:")
    
    # Identifico le domande mancanti
    section_nums = []
    for q in section_questions:
        parts = q.split('.')
        if len(parts) >= 2:
            section_nums.append(int(parts[1]))
    
    if section_nums:
        min_num = min(section_nums)
        max_num = max(section_nums)
        missing = []
        
        for i in range(min_num, max_num + 1):
            expected_q = f"{section}.{i}"
            if expected_q not in section_questions:
                missing.append(expected_q)
        
        print(f"  ✓ Presenti ({len(section_questions)}): {', '.join(section_questions)}")
        if missing:
            print(f"  ❌ MANCANTI ({len(missing)}): {', '.join(missing)}")

# Riepilogo domande mancanti
print("\n" + "="*60)
print("RIEPILOGO DOMANDE MANCANTI:")
all_missing = []
for section in sorted(sections.keys(), key=int):
    section_questions = sorted(sections[section], key=lambda x: [int(n) for n in x.split('.')])
    section_nums = []
    for q in section_questions:
        parts = q.split('.')
        if len(parts) >= 2:
            section_nums.append(int(parts[1]))
    
    if section_nums:
        min_num = min(section_nums)
        max_num = max(section_nums)
        
        for i in range(min_num, max_num + 1):
            expected_q = f"{section}.{i}"
            if expected_q not in section_questions:
                all_missing.append(expected_q)

if all_missing:
    print(f"Totale domande mancanti: {len(all_missing)}")
    for missing_q in all_missing:
        print(f"  - {missing_q}")
else:
    print("Nessuna domanda mancante nella numerazione sequenziale!")

Tentativo di caricamento: ./dati backup/cpia.xlsx
File caricato correttamente! Shape: (5, 260)
File utilizzato: ./dati backup/cpia.xlsx
=== ANALISI COMPLETA DELLE COLONNE NEL DATASET ===
Numero totale di colonne: 260

Domande trovate nel dataset (47):
 1. 1.1 (2 colonne)
 2. 1.2 (1 colonne)
 3. 1.3 (1 colonne)
 4. 1.4 (5 colonne)
 5. 1.5 (1 colonne)
 6. 1.6 (1 colonne)
 7. 1.7 (2 colonne)
 8. 1.8 (1 colonne)
 9. 1.9 (1 colonne)
10. 2.1 (1 colonne)
11. 2.2 (5 colonne)
12. 2.3 (4 colonne)
13. 2.4 (4 colonne)
14. 2.5 (4 colonne)
15. 2.6 (3 colonne)
16. 2.7 (5 colonne)
17. 2.8 (1 colonne)
18. 3.1 (1 colonne)
19. 3.2 (15 colonne)
20. 3.3 (3 colonne)
21. 3.4 (7 colonne)
22. 3.5 (6 colonne)
23. 3.6 (1 colonne)
24. 3.7 (1 colonne)
25. 3.8 (3 colonne)
26. 3.9 (6 colonne)
27. 3.10 (7 colonne)
28. 3.11 (4 colonne)
29. 3.12 (1 colonne)
30. 3.13 (5 colonne)
31. 3.14 (1 colonne)
32. 3.15 (7 colonne)
33. 3.16 (1 colonne)
34. 4.1 (1 colonne)
35. 4.2 (7 colonne)
36. 4.3 (6 colonne)
37. 4.4 (6 colonne)


# Pipeline questionario: merge, analisi, selezione colonne
Questo notebook esegue in sequenza:
1) Unione dei file Excel e aggiunta del numero file
2) Analisi delle intestazioni e coperture
3) Selezione delle colonne utili e creazione dataset ridotto

In [9]:
# 1) Merge dei file Excel dalla cartella "dati backup" (sotto la cartella del notebook)
import os, pandas as pd, re

base_dir = os.getcwd()
input_dir = os.path.join(base_dir, "dati backup")
pattern = re.compile(r"results-survey(\d+)\.xlsx")

if not os.path.isdir(input_dir):
    raise FileNotFoundError(f"Cartella non trovata: {input_dir}")

dfs = []
for fn in os.listdir(input_dir):
    m = pattern.match(fn)
    if m:
        num = m.group(1)
        full_path = os.path.join(input_dir, fn)
        df = pd.read_excel(full_path)
        df["file_number"] = num
        dfs.append(df)

merged = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
merged.to_excel("merged_results.xlsx", index=False)
len(merged), merged.columns.tolist()[:10]

(0, [])

In [10]:
# Analisi della struttura delle colonne per identificare domande e opzioni
import re
from collections import defaultdict

# Carica il file merged per analizzare la struttura
df_analysis = pd.read_excel('merged_results.xlsx')

print(f"Totale colonne: {len(df_analysis.columns)}")
print(f"Totale righe: {len(df_analysis)}")

# Identifica pattern per domande con opzioni
question_to_options = defaultdict(list)
base_questions = set()

for col in df_analysis.columns:
    # Pattern per domande con opzioni in parentesi quadre: "domanda [opzione]"
    if '[' in col and ']' in col:
        # Estrae la domanda base (tutto prima della prima [)
        base_question = col.split('[')[0].strip()
        if base_question.endswith(':'):
            base_question = base_question[:-1].strip()
        # Estrae l'opzione (tutto tra [ e ])
        option_match = re.search(r'\[([^\]]+)\]', col)
        if option_match:
            option = option_match.group(1).strip()
            question_to_options[base_question].append({
                'option': option,
                'full_column': col
            })
    else:
        # Domanda semplice senza opzioni
        clean_col = col.strip()
        if clean_col.endswith(':'):
            clean_col = clean_col[:-1].strip()
        base_questions.add(clean_col)

print(f"\nDomande con opzioni: {len(question_to_options)}")
print(f"Domande semplici: {len(base_questions)}")

# Mostra alcuni esempi
print("\n=== ESEMPI DI DOMANDE CON OPZIONI ===")
for i, (base_q, options) in enumerate(list(question_to_options.items())[:5]):
    print(f"\n{i+1}. {base_q}")
    for opt in options[:3]:  # Mostra solo le prime 3 opzioni
        print(f"   - {opt['option']} (colonna: {opt['full_column']})")
    if len(options) > 3:
        print(f"   ... e altre {len(options)-3} opzioni")

print("\n=== ESEMPI DI DOMANDE SEMPLICI ===")
simple_questions = [q for q in base_questions if any(q.startswith(p) for p in ['1.', '2.', '3.', '4.', '5.', '6.'])][:5]
for i, q in enumerate(simple_questions):
    print(f"{i+1}. {q}")

Totale colonne: 0
Totale righe: 0

Domande con opzioni: 0
Domande semplici: 0

=== ESEMPI DI DOMANDE CON OPZIONI ===

=== ESEMPI DI DOMANDE SEMPLICI ===


In [11]:
# 2) Analisi intestazioni e copertura + Estrazione opzioni di risposta CORRETTA
import pandas as pd, unicodedata, re
from collections import defaultdict

def remove_diacritics(s):
    nfkd = unicodedata.normalize('NFKD', str(s));
    return ''.join(ch for ch in nfkd if not unicodedata.combining(ch))
def normalize_name(s):
    s=str(s); s=remove_diacritics(s).lower().strip();
    s=re.sub(r'\s+',' ',s); s=re.sub(r'[^0-9a-z]','',s); return s

# === Opzioni teoriche complete per domande specifiche ===
KNOWN_QUESTION_OPTIONS = {
    '1.1 Ruolo': [
        'docente',
        'docente referente per l\'orientamento e l\'accoglienza', 
        'Altro'
    ]
    # Altre domande con opzioni note possono essere aggiunte qui
}

# Colonne di metadati da escludere dalla classificazione come domande
METADATA_COLUMNS = {
    'ID risposta', 'Data invio', 'Ultima pagina', 'Lingua iniziale', 'Seme', 
    'Data di inizio', 'Data dellultima azione', 'file_number'
}

# Carica i dati dal file corretto (non dal merged_results.xlsx vuoto)
try:
    df = pd.read_excel('./dati backup/cpia.xlsx')
    print(f"Dataset caricato correttamente: {df.shape}")
except FileNotFoundError:
    try:
        df = pd.read_excel('merged_results.xlsx')
        print(f"Dataset caricato da merged_results.xlsx: {df.shape}")
    except:
        print("ERRORE: Impossibile caricare alcun dataset!")
        df = pd.DataFrame()

if df.empty:
    print("ATTENZIONE: Dataset vuoto! Impossibile procedere con l'analisi.")
else:
    print(f"Colonne trovate: {len(df.columns)}")
    print(f"Prime 5 colonne: {df.columns[:5].tolist()}")

# === Analisi intestazioni originale ===
rows = []
for c in df.columns:
    s = df[c]
    rows.append({
        'original_name': c,
        'normalized_name': normalize_name(c),
        'dtype': str(s.dtype),
        'non_null_count': int(s.notna().sum()),
        'unique_count_non_null': s.nunique(),
        'unique_count_including_na': len(s.unique()),
        'example_values': str(s.dropna().head(3).tolist())[:200] if not s.dropna().empty else ''
    })

ha = pd.DataFrame(rows)
ha.to_csv('header_analysis.csv', index=False)

# === Estrazione opzioni di risposta migliorata ===
question_to_options = defaultdict(list)
all_base_questions = set()

# Pattern per individuare domande con opzioni [opzione]
pattern = re.compile(r'^(.*?)\s*\[(.*?)\]\s*:?\s*$')

for col in df.columns:
    # Salta le colonne di metadati
    if col in METADATA_COLUMNS:
        continue
        
    match = pattern.match(col)
    if match:
        base_question = match.group(1).strip()
        option = match.group(2).strip()
        all_base_questions.add(base_question)
        question_to_options[base_question].append({
            'option': option,
            'full_column': col,
            'is_base_value': False
        })
    else:
        # Controlla se è una colonna base (senza opzioni) ma solo per domande numerate
        clean_col = col.strip().rstrip(':').strip()
        
        # Verifica se è una domanda numerata (inizia con numero.numero)
        question_pattern = re.compile(r'^(\d+\.\d+)')
        if question_pattern.match(clean_col):
            # Verifica se esiste una versione con opzioni di questa colonna
            questions_with_base_columns = set()
            for base_q in all_base_questions:
                if clean_col.startswith(base_q) or base_q.startswith(clean_col):
                    questions_with_base_columns.add(base_q)
            
            # Se non ha opzioni corrispondenti nella forma [opzione], 
            # potrebbe essere una colonna base con valori diretti
            if not questions_with_base_columns:
                # Controllo se la colonna ha valori che sembrano opzioni multiple
                unique_values = df[col].dropna().unique()
                if len(unique_values) > 0:
                    # Se i valori contengono separatori come ";" potrebbe essere multiple choice
                    has_separators = any(';' in str(val) or ',' in str(val) for val in unique_values)
                    if has_separators or (len(unique_values) <= 10 and len(unique_values) > 1):
                        question_to_options[clean_col].append({
                            'option': 'base_values',
                            'full_column': col,
                            'is_base_value': True
                        })

# === Processo per creare il mapping completo domande-opzioni ===
questions_data = []

for base_q, options in question_to_options.items():
    if not options:
        continue
    
    columns_for_question = []
    all_options_for_question = []
    
    # Se abbiamo opzioni teoriche definite, usale
    if base_q in KNOWN_QUESTION_OPTIONS:
        theoretical_options = KNOWN_QUESTION_OPTIONS[base_q]
        all_options_for_question = theoretical_options.copy()
        
        # Aggiungi i valori "Altro: specificazione" se esistono colonne Altro
        for opt_info in options:
            if 'altro' in opt_info['option'].lower():
                specific_col = opt_info['full_column']
                if specific_col in df.columns:
                    specific_values = df[specific_col].dropna().unique().tolist()
                    if specific_values:
                        # Sostituisci "Altro" con "Altro: valore_specifico"
                        for i, opt in enumerate(all_options_for_question):
                            if opt.lower() == 'altro':
                                all_options_for_question[i] = f"Altro: {'; '.join(specific_values)}"
    else:
        # Logica originale per domande senza opzioni teoriche definite
        for opt_info in options:
            if opt_info.get('is_base_value', False):
                # Opzione dalla colonna base - estrai i valori unici
                unique_values = df[opt_info['full_column']].dropna().unique()
                all_options_for_question.extend([str(val) for val in unique_values])
                if opt_info['full_column'] not in columns_for_question:
                    columns_for_question.append(opt_info['full_column'])
            else:
                # Opzione da colonna specifica [opzione]
                if opt_info['option'].lower() == 'altro':
                    specific_col = opt_info['full_column']
                    specific_values = df[specific_col].dropna().unique().tolist()
                    if specific_values:
                        for spec_val in specific_values:
                            all_options_for_question.append(f"Altro: {spec_val}")
                    else:
                        all_options_for_question.append(opt_info['option'])
                else:
                    all_options_for_question.append(opt_info['option'])
                columns_for_question.append(opt_info['full_column'])
    
    # Raccogli tutte le colonne per questa domanda
    for opt_info in options:
        if opt_info['full_column'] not in columns_for_question:
            columns_for_question.append(opt_info['full_column'])
    
    # Rimuovi duplicati mantenendo l'ordine
    unique_options = []
    seen_options = set()
    for opt in all_options_for_question:
        if opt not in seen_options:
            unique_options.append(opt)
            seen_options.add(opt)
    
    all_options_str = '; '.join(unique_options)
    
    # Crea una riga per ogni colonna, ma tutte con lo stesso set di opzioni
    for col in columns_for_question:
        questions_data.append({
            'question_base': base_q,
            'question_type': 'multiple_choice',
            'full_column': col,
            'option': None,  # Le opzioni sono in 'all_options'
            'options_count': len(unique_options),
            'all_options': all_options_str
        })

# Gestisci le domande semplici (senza opzioni) - solo quelle numerate
for col in df.columns:
    # Salta metadati
    if col in METADATA_COLUMNS:
        continue
        
    if '[' not in col and ']' not in col:
        clean_col = col.strip()
        if clean_col.endswith(':'):
            clean_col = clean_col[:-1].strip()
        
        # Verifica che sia una domanda numerata
        question_pattern = re.compile(r'^(\d+\.\d+)')
        if question_pattern.match(clean_col):
            # Se non ha opzioni corrispondenti, è una domanda semplice
            if clean_col not in all_base_questions:
                questions_data.append({
                    'question_base': clean_col,
                    'question_type': 'simple',
                    'full_column': col,
                    'option': None,
                    'options_count': 0,
                    'all_options': ''
                })

# Salva il mapping completo delle domande e opzioni
questions_df = pd.DataFrame(questions_data)
questions_df.to_csv('questions_with_options.csv', index=False)

print(f"Salvato file 'questions_with_options.csv' con {len(questions_df)} righe")
print(f"Domande semplici: {len([q for q in questions_data if q['question_type'] == 'simple'])}")
print(f"Righe di domande multiple: {len([q for q in questions_data if q['question_type'] == 'multiple_choice'])}")
print(f"Domande multiple uniche: {len(set(q['question_base'] for q in questions_data if q['question_type'] == 'multiple_choice'))}")

# === Copertura per file (codice originale) ===
if 'file_number' in df.columns:
    long = []
    for fn, g in df.groupby('file_number', dropna=False):
        n=len(g)
        for c in df.columns:
            if c=='file_number': continue
            non_null=int(g[c].notna().sum()); pct=round(100*non_null/max(n,1),2)
            long.append({'file_number':fn,'column':c,'non_null_pct':pct,'non_null_count':non_null,'rows_in_file':n})
    pd.DataFrame(long).to_csv('header_coverage_by_file.csv', index=False)

# Mostra esempi di domande con opzioni (correzione delle domande problematiche)
print("\n=== VERIFICA DOMANDA 1.1 RUOLO ===")
if len(questions_df) > 0 and 'question_base' in questions_df.columns:
    ruolo_rows = questions_df[questions_df['question_base'] == '1.1 Ruolo']
    if not ruolo_rows.empty:
        print(f"Tipo: {ruolo_rows.iloc[0]['question_type']}")
        print(f"Opzioni: {ruolo_rows.iloc[0]['all_options']}")
        print(f"Colonne coinvolte: {ruolo_rows['full_column'].tolist()}")
    else:
        print("ERRORE: Domanda 1.1 Ruolo non trovata!")
else:
    print("ERRORE: DataFrame questions_df vuoto o senza colonna question_base!")

print(f"\n=== ANALISI COMPLETATA ===")
print(f"File salvati:")
print(f"- header_analysis.csv: {len(ha)} righe")
print(f"- questions_with_options.csv: {len(questions_df)} righe")

Dataset caricato correttamente: (5, 260)
Colonne trovate: 260
Prime 5 colonne: ['ID risposta', 'Data invio', 'Ultima pagina', 'Lingua iniziale', 'Seme']
Salvato file 'questions_with_options.csv' con 204 righe
Domande semplici: 20
Righe di domande multiple: 184
Domande multiple uniche: 41

=== VERIFICA DOMANDA 1.1 RUOLO ===
Tipo: multiple_choice
Opzioni: docente; docente referente per l'orientamento e l'accoglienza; Altro
Colonne coinvolte: ['1.1 Ruolo:', '1.1 Ruolo:']

=== ANALISI COMPLETATA ===
File salvati:
- header_analysis.csv: 260 righe
- questions_with_options.csv: 204 righe


In [12]:
# Verifica il contenuto del file delle domande con opzioni
try:
    questions_info = pd.read_csv('questions_with_options.csv')
    if len(questions_info) > 0:
        print("Colonne del file 'questions_with_options.csv':")
        print(questions_info.columns.tolist())
        print(f"\nPrime 5 righe:")
        print(questions_info.head())
        print(f"\nStatistiche:")
        print(f"- Righe totali: {len(questions_info)}")
        print(f"- Domande uniche: {questions_info['question_base'].nunique()}")
        print(f"- Domande semplici: {len(questions_info[questions_info['question_type'] == 'simple'])}")
        print(f"- Opzioni di domande multiple: {len(questions_info[questions_info['question_type'] == 'multiple_choice'])}")
    else:
        print("Il file 'questions_with_options.csv' è vuoto!")
except FileNotFoundError:
    print("Il file 'questions_with_options.csv' non esiste!")
except pd.errors.EmptyDataError:
    print("Il file 'questions_with_options.csv' esiste ma è vuoto o corrotto!")
except Exception as e:
    print(f"Errore nel caricamento del file: {e}")

Colonne del file 'questions_with_options.csv':
['question_base', 'question_type', 'full_column', 'option', 'options_count', 'all_options']

Prime 5 righe:
                                       question_base    question_type  \
0                                          1.1 Ruolo  multiple_choice   
1                                         1.1 Ruolo:  multiple_choice   
2                                         1.3 Genere  multiple_choice   
3  1.4 Titolo di studio (indichi tutti i titoli p...  multiple_choice   
4  1.4 Titolo di studio (indichi tutti i titoli p...  multiple_choice   

                                         full_column  option  options_count  \
0                                         1.1 Ruolo:     NaN              3   
1                                 1.1 Ruolo: [Altro]     NaN              1   
2                                        1.3 Genere:     NaN              2   
3  1.4 Titolo di studio (indichi tutti i titoli p...     NaN              5   
4  1.4 Tito

In [13]:
# 3) Selezione colonne utili e dataset ridotto CON OPZIONI DI RISPOSTA
import pandas as pd, re
OPEN_TEXT_KEYWORDS = [
    'Specificare', 'Quali?', 'Quali ', 'Scriva', 'A quali', 'Indichi i corsi',
    'titolo del corso', 'ente organizzatore', 'tipologia (master', 'durata (indicare in ore)',
    'anno di conseguimento', 'Altro]'
]
META_EXACT = { 'ID risposta','Data invio','Ultima pagina','Lingua iniziale','Seme','Data di inizio','Data dellultima azione','file_number'}
EXCLUDE_PREFIXES = ['Tempo totale','Tempo per il gruppo di domande','Tempo per la domanda']
TITLE_OF_STUDY_WHITELIST = [
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [diploma]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [laurea]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di perfezionamento ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di specializzazione ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [master ]',
    '1.4 Titolo di studio (indichi tutti i titoli posseduti): [dottorato ]',
]
KEEP_PREFIXES = ['1.','2.','3.','4.','5.','6.','7.']
KEEP_STRICT_PREFIXES = ['3.17 ']

# Carica i dati di analisi delle intestazioni e delle opzioni
ha = pd.read_csv('header_analysis.csv')
questions_info = pd.read_csv('questions_with_options.csv')

candidates = []
def is_open_text(name:str)->bool:
    nl=name.lower();
    return any(k.lower() in nl for k in OPEN_TEXT_KEYWORDS)

for _,row in ha.iterrows():
    name = str(row['original_name'])
    if name in META_EXACT or any(name.startswith(p) for p in EXCLUDE_PREFIXES):
        continue
    non_null = int(row.get('non_null_count',0))
    unique_cnt = int(row.get('unique_count_non_null',0)) if pd.notna(row.get('unique_count_non_null',0)) else 0
    if non_null==0: continue
    if is_open_text(name) and unique_cnt>20: continue
    if name in TITLE_OF_STUDY_WHITELIST: candidates.append(name); continue
    if any(name.startswith(p) for p in KEEP_STRICT_PREFIXES): candidates.append(name); continue
    if any(name.startswith(p) for p in KEEP_PREFIXES):
        if is_open_text(name) and unique_cnt>20: continue
        candidates.append(name)

keep=[]; seen=set()
for c in candidates:
    if c not in seen:
        keep.append(c); seen.add(c)

print(f"PRIMA DEL FIX - Colonne selezionate dal filtro originale: {len(keep)}")

# NUOVO: Aggiungi TUTTE le colonne che iniziano con numeri (1., 2., 3., etc) al keep
# anche se non passano i filtri originali, così includiamo tutte le domande
merged = pd.read_excel('./dati backup/cpia.xlsx')  # Usa file con dati reali
all_question_columns = []
pattern = r'^(\d+\.\d+(?:\.\d+)?)'

for col in merged.columns:
    match = re.match(pattern, col)
    if match:
        all_question_columns.append(col)

# Combina i keep originali con tutte le colonne domanda
keep_extended = list(set(keep + all_question_columns))
print(f"DOPO IL FIX - Colonne totali (originali + tutte le domande): {len(keep_extended)}")
print(f"Domande aggiunte: {len(keep_extended) - len(keep)}")

# Crea il dataset ridotto con TUTTE le domande
subset = merged[[c for c in keep_extended if c in merged.columns]]
subset.to_csv('dataset_domande.csv', index=False)

# Crea un file aggiuntivo con informazioni sulle domande e opzioni per TUTTE le colonne domanda
selected_questions_info = questions_info[questions_info['full_column'].isin(keep_extended)].copy()

# Crea un riassunto per domanda base con tutte le opzioni
questions_summary = []
for question_base in selected_questions_info['question_base'].unique():
    q_data = selected_questions_info[selected_questions_info['question_base'] == question_base]
    
    if q_data.iloc[0]['question_type'] == 'simple':
        # Domanda semplice
        questions_summary.append({
            'question_base': question_base,
            'question_type': 'simple',
            'columns_count': 1,
            'columns': q_data['full_column'].tolist()[0],
            'options_count': 0,
            'all_options': '',
            'sample_columns': q_data['full_column'].tolist()[:5]  # Per compatibilità
        })
    else:
        # Domanda con opzioni multiple
        all_options = q_data['all_options'].iloc[0]  # È uguale per tutte le righe della stessa domanda
        columns_list = q_data['full_column'].tolist()
        options_list = q_data['option'].tolist()
        
        # Conta le opzioni dal campo all_options (più accurato)
        options_count = len(all_options.split('; ')) if all_options else 0
        
        questions_summary.append({
            'question_base': question_base,
            'question_type': 'multiple_choice',
            'columns_count': len(columns_list),
            'columns': '; '.join(columns_list),
            'options_count': options_count,
            'all_options': all_options,
            'sample_columns': columns_list[:5]  # Prime 5 colonne se ce ne sono molte
        })

# Salva il riassunto delle domande selezionate
questions_summary_df = pd.DataFrame(questions_summary)
questions_summary_df.to_csv('dataset_domande_con_opzioni.csv', index=False)

# Salva anche il dettaglio completo delle domande selezionate
selected_questions_info.to_csv('dataset_domande_dettaglio.csv', index=False)

print(f"Dataset creato con {len(keep_extended)} colonne selezionate")
print("File salvati:")
print(f"  - dataset_domande.csv (dati): {subset.shape}")
print(f"  - dataset_domande_con_opzioni.csv (riassunto domande): {questions_summary_df.shape}")
print(f"  - dataset_domande_dettaglio.csv (dettaglio completo): {selected_questions_info.shape}")

print("\n=== DOMANDE SELEZIONATE CON LE LORO OPZIONI ===")
for i, row in questions_summary_df.iterrows():
    q_type = "multiple_choice" if row['question_type'] == 'multiple_choice' else 'simple'
    if q_type == 'multiple_choice':
        print(f"{i+1:2d}. {row['question_base']} ({q_type})")
        print(f"    Opzioni ({row['options_count']}): {row['all_options']}")
        print(f"    Colonne: {row['columns_count']}")
    else:
        print(f"{i+1:2d}. {row['question_base']} ({q_type})")
        print(f"    Colonna: {row['columns']}")

print(f"\n=== STATISTICHE FINALI ===")
print(f"Domande totali: {len(questions_summary_df)}")
print(f"Domande multiple choice: {len(questions_summary_df[questions_summary_df['question_type'] == 'multiple_choice'])}")
print(f"Domande semplici: {len(questions_summary_df[questions_summary_df['question_type'] == 'simple'])}")

PRIMA DEL FIX - Colonne selezionate dal filtro originale: 181
DOPO IL FIX - Colonne totali (originali + tutte le domande): 199
Domande aggiunte: 18
Dataset creato con 199 colonne selezionate
File salvati:
  - dataset_domande.csv (dati): (5, 199)
  - dataset_domande_con_opzioni.csv (riassunto domande): (49, 7)
  - dataset_domande_dettaglio.csv (dettaglio completo): (204, 6)

=== DOMANDE SELEZIONATE CON LE LORO OPZIONI ===
 1. 1.1 Ruolo (multiple_choice)
    Opzioni (3): docente; docente referente per l'orientamento e l'accoglienza; Altro
    Colonne: 2
 2. 1.1 Ruolo: (multiple_choice)
    Opzioni (1): Altro: A022
    Colonne: 1
 3. 1.3 Genere (multiple_choice)
    Opzioni (2): femmina; maschio
    Colonne: 2
 4. 1.4 Titolo di studio (indichi tutti i titoli posseduti): (multiple_choice)
    Opzioni (5): laurea; corsi di perfezionamento; corsi di specializzazione; master; dottorato
    Colonne: 5
 5. 1.5 Anzianità di servizio non di ruolo nella scuola (multiple_choice)
    Opzioni (2): ol

In [14]:
# Verifica le domande multiple choice selezionate
questions_summary_df = pd.read_csv('dataset_domande_con_opzioni.csv')
multiple_choice_questions = questions_summary_df[questions_summary_df['question_type'] == 'multiple_choice']

print(f"Domande con opzioni multiple selezionate: {len(multiple_choice_questions)}")
print("\n=== TUTTE LE DOMANDE CON OPZIONI MULTIPLE ===")
for i, row in multiple_choice_questions.iterrows():
    print(f"{i+1:2d}. {row['question_base']}")
    print(f"    Opzioni ({row['options_count']}): {row['all_options']}")
    print(f"    Colonne nel dataset: {row['columns_count']}")
    print()

# Verifica i file creati
import os
files_created = [
    'dataset_domande.csv',
    'dataset_domande_con_opzioni.csv', 
    'dataset_domande_dettaglio.csv',
    'questions_with_options.csv',
    'header_analysis.csv'
]

print("=== FILE CREATI ===")
for file in files_created:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"✓ {file} ({size} bytes)")
    else:
        print(f"✗ {file} (non trovato)")

Domande con opzioni multiple selezionate: 41

=== TUTTE LE DOMANDE CON OPZIONI MULTIPLE ===
 1. 1.1 Ruolo
    Opzioni (3): docente; docente referente per l'orientamento e l'accoglienza; Altro
    Colonne nel dataset: 2

 2. 1.1 Ruolo:
    Opzioni (1): Altro: A022
    Colonne nel dataset: 1

 3. 1.3 Genere
    Opzioni (2): femmina; maschio
    Colonne nel dataset: 2

 4. 1.4 Titolo di studio (indichi tutti i titoli posseduti):
    Opzioni (5): laurea; corsi di perfezionamento; corsi di specializzazione; master; dottorato
    Colonne nel dataset: 5

 5. 1.5 Anzianità di servizio non di ruolo nella scuola
    Opzioni (2): oltre 15 anni; meno di 5 anni
    Colonne nel dataset: 2

 6. 1.6 Anzianità di servizio di ruolo nella scuola
    Opzioni (2): oltre 15 anni; meno di 5 anni
    Colonne nel dataset: 2

 7. 1.7 Attualmente svolge altri incarichi (vicepreside, funzione strumentale etc.)?
    Opzioni (2): No; Sì
    Colonne nel dataset: 2

 8. 1.8 Per quale classe di concorso ha conseguito 

In [49]:
# Debug: controllo specifico della domanda "Ruolo"
df_check = pd.read_excel('merged_results.xlsx')

print("=== CONTROLLO DOMANDA RUOLO ===")
ruolo_columns = [col for col in df_check.columns if 'ruolo' in col.lower() or 'Ruolo' in col]
print(f"Colonne contenenti 'Ruolo': {len(ruolo_columns)}")
for col in ruolo_columns:
    print(f"  - {col}")

print("\n=== VALORI NELLA COLONNA '1.1 Ruolo:' ===")
if '1.1 Ruolo:' in df_check.columns:
    ruolo_values = df_check['1.1 Ruolo:'].dropna().unique()
    print(f"Valori unici ({len(ruolo_values)}): {ruolo_values}")

print("\n=== VALORI NELLA COLONNA '1.1 Ruolo: [Altro]' ===")
if '1.1 Ruolo: [Altro]' in df_check.columns:
    altro_values = df_check['1.1 Ruolo: [Altro]'].dropna().unique()
    print(f"Valori unici ({len(altro_values)}): {altro_values}")

# Verifica come è stata processata nel mio algoritmo
print("\n=== COME È STATA PROCESSATA ===")
questions_info_check = pd.read_csv('questions_with_options.csv')
ruolo_rows = questions_info_check[questions_info_check['question_base'].str.contains('Ruolo', na=False)]
print(ruolo_rows[['question_base', 'question_type', 'full_column', 'option']])

=== CONTROLLO DOMANDA RUOLO ===
Colonne contenenti 'Ruolo': 0

=== VALORI NELLA COLONNA '1.1 Ruolo:' ===

=== VALORI NELLA COLONNA '1.1 Ruolo: [Altro]' ===

=== COME È STATA PROCESSATA ===
    question_base    question_type         full_column  option
0       1.1 Ruolo  multiple_choice          1.1 Ruolo:     NaN
1      1.1 Ruolo:  multiple_choice  1.1 Ruolo: [Altro]     NaN
184     1.1 Ruolo           simple          1.1 Ruolo:     NaN


In [4]:
# Correzione: identifico tutti i valori per la domanda Ruolo
print("=== ANALISI COMPLETA DOMANDA 1.1 RUOLO ===")

# Utilizziamo il DataFrame che abbiamo già caricato correttamente
if '1.1 Ruolo:' in df.columns:
    # Valori dalla colonna principale
    main_ruolo_values = df['1.1 Ruolo:'].dropna().unique().tolist()
    print(f"Valori da '1.1 Ruolo:': {main_ruolo_values}")
    
    # Se esiste "Altro" come valore, significa che nella colonna [Altro] ci sono le specifiche
    altro_specific_values = []
    if '1.1 Ruolo: [Altro]' in df.columns:
        altro_specific_values = df['1.1 Ruolo: [Altro]'].dropna().unique().tolist()
        print(f"Valori specifici da '1.1 Ruolo: [Altro]': {altro_specific_values}")
    
    # La lista completa delle opzioni dovrebbe essere:
    all_ruolo_options = []
    for val in main_ruolo_values:
        if val != 'Altro':  # Aggiungi valori non-altro direttamente
            all_ruolo_options.append(val)
    
    # Aggiungi i valori specifici di "Altro" 
    if altro_specific_values:
        all_ruolo_options.extend([f"Altro: {val}" for val in altro_specific_values])
        
    print(f"\nTutte le opzioni per 1.1 Ruolo: {all_ruolo_options}")
    
    # Verifica cosa dovrebbe succedere:
    print("\n=== COSA DOVREBBE SUCCEDERE ===")
    print("La domanda '1.1 Ruolo' dovrebbe essere classificata come 'multiple_choice' con opzioni:")
    for i, opt in enumerate(all_ruolo_options, 1):
        print(f"  {i}. {opt}")
        
    print("\nE dovrebbero esserci 2 colonne nel dataset:")
    print("  - 1.1 Ruolo: (per i valori 'docente', 'Altro')")
    print("  - 1.1 Ruolo: [Altro] (per le specifiche quando è selezionato 'Altro')")
else:
    print("ERRORE: La colonna '1.1 Ruolo:' non esiste nel DataFrame!")
    print("Colonne che iniziano con '1.1':")
    for col in df.columns:
        if col.startswith('1.1'):
            print(f"  - {col}")

=== ANALISI COMPLETA DOMANDA 1.1 RUOLO ===
Valori da '1.1 Ruolo:': ['docente', 'Altro']
Valori specifici da '1.1 Ruolo: [Altro]': ['A022']

Tutte le opzioni per 1.1 Ruolo: ['docente', 'Altro: A022']

=== COSA DOVREBBE SUCCEDERE ===
La domanda '1.1 Ruolo' dovrebbe essere classificata come 'multiple_choice' con opzioni:
  1. docente
  2. Altro: A022

E dovrebbero esserci 2 colonne nel dataset:
  - 1.1 Ruolo: (per i valori 'docente', 'Altro')
  - 1.1 Ruolo: [Altro] (per le specifiche quando è selezionato 'Altro')


In [54]:
# === VERIFICA DOMANDE MANCANTI (1.2 e altre) ===
print("=== VERIFICA DOMANDE MANCANTI ===")

# Cerco tutte le colonne che iniziano con numeri per identificare le domande
question_pattern = re.compile(r'^(\d+\.\d+)')
all_question_bases = set()

for col in df.columns:
    match = question_pattern.match(col)
    if match:
        question_num = match.group(1)
        all_question_bases.add(question_num)

print(f"Domande trovate nel dataset ({len(all_question_bases)}):")
for q in sorted(all_question_bases):
    print(f"  - {q}")

# Verifica specificamente domande come 1.2, 2.3, 3.2, etc.
expected_questions = ['1.1', '1.2', '2.1', '2.2', '2.3', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7']
missing_questions = []
for q in expected_questions:
    if q not in all_question_bases:
        missing_questions.append(q)

if missing_questions:
    print(f"\nDomande che sembrano mancare: {missing_questions}")
else:
    print("\nTutte le domande attese sono presenti!")

# Verifica specifica per 1.2
print(f"\n=== RICERCA SPECIFICA PER DOMANDA 1.2 ===")
cols_1_2 = [col for col in df.columns if '1.2' in col]
if cols_1_2:
    print(f"Colonne trovate con '1.2': {cols_1_2}")
else:
    print("Nessuna colonna trovata con '1.2'")
    
# Mostra le prime colonne per capire il pattern
print(f"\n=== PRIME 20 COLONNE DEL DATASET ===")
for i, col in enumerate(df.columns[:20]):
    print(f"{i+1:2d}. {col}")

=== VERIFICA DOMANDE MANCANTI ===
Domande trovate nel dataset (47):
  - 1.1
  - 1.2
  - 1.3
  - 1.4
  - 1.5
  - 1.6
  - 1.7
  - 1.8
  - 1.9
  - 2.1
  - 2.2
  - 2.3
  - 2.4
  - 2.5
  - 2.6
  - 2.7
  - 2.8
  - 3.1
  - 3.10
  - 3.11
  - 3.12
  - 3.13
  - 3.14
  - 3.15
  - 3.16
  - 3.2
  - 3.3
  - 3.4
  - 3.5
  - 3.6
  - 3.7
  - 3.8
  - 3.9
  - 4.1
  - 4.2
  - 4.3
  - 4.4
  - 4.5
  - 4.6
  - 4.7
  - 5.1
  - 5.2
  - 6.1
  - 6.2
  - 6.3
  - 6.4
  - 7.1

Tutte le domande attese sono presenti!

=== RICERCA SPECIFICA PER DOMANDA 1.2 ===
Colonne trovate con '1.2': ['1.2 Età:']

=== PRIME 20 COLONNE DEL DATASET ===
 1. ID risposta
 2. Data invio
 3. Ultima pagina
 4. Lingua iniziale
 5. Seme
 6. 1.1 Ruolo:
 7. 1.1 Ruolo: [Altro]
 8. 1.2 Età:
 9. 1.3 Genere:
10. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [laurea ]
11. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di perfezionamento ]
12. 1.4 Titolo di studio (indichi tutti i titoli posseduti): [corsi di specializzaz

In [55]:
# Definisco le opzioni teoriche corrette per domande specifiche
KNOWN_QUESTION_OPTIONS = {
    '1.1 Ruolo': [
        'docente',
        'docente referente per l\'orientamento e l\'accoglienza', 
        'Altro'
    ]
    # Posso aggiungere altre domande con opzioni note qui
}

print("=== CORREZIONE NECESSARIA ===")
print("Le opzioni attuali sono incomplete.")
print("Devo modificare l'algoritmo per includere le opzioni teoriche complete.")
print("\nPer '1.1 Ruolo' le opzioni corrette sono:")
for i, opt in enumerate(KNOWN_QUESTION_OPTIONS['1.1 Ruolo'], 1):
    print(f"  {i}. {opt}")
    
print(f"\nE quando qualcuno sceglie 'Altro', la specifica è: {ruolo_altro_values[0]}")
print("Quindi l'opzione finale dovrebbe essere: 'Altro: A022'")

=== CORREZIONE NECESSARIA ===
Le opzioni attuali sono incomplete.
Devo modificare l'algoritmo per includere le opzioni teoriche complete.

Per '1.1 Ruolo' le opzioni corrette sono:
  1. docente
  2. docente referente per l'orientamento e l'accoglienza
  3. Altro

E quando qualcuno sceglie 'Altro', la specifica è: A022
Quindi l'opzione finale dovrebbe essere: 'Altro: A022'


In [56]:
# === DIAGNOSI DOMANDA 1.2 AGE ===
print("=== DIAGNOSI PERCHÉ LA DOMANDA 1.2 NON È NELL'EXPORT ===")

# Cerco le colonne 1.2
cols_1_2 = [col for col in df.columns if col.startswith('1.2')]
print(f"Colonne che iniziano con '1.2': {cols_1_2}")

if '1.2 Età:' in df.columns:
    print(f"\nValori unici in '1.2 Età:': {df['1.2 Età:'].dropna().unique()}")
    print(f"Tipo di valori: {type(df['1.2 Età:'].dropna().iloc[0] if len(df['1.2 Età:'].dropna()) > 0 else 'N/A')}")
    
    # Controlla se viene riconosciuta come domanda base
    base_questions_current = set()
    for col in df.columns:
        if col.startswith('1.2'):
            match = re.match(r'^(.*?)(?:\s*\[.*?\]\s*)?:?\s*$', col)
            if match:
                base_questions_current.add(match.group(1))
    
    print(f"\nBase questions per 1.2 trovate: {base_questions_current}")
    
    # Verifica se è classificata come simple o multiple choice
    question_base = '1.2 Età'
    columns_for_question = [col for col in df.columns if col.startswith(question_base)]
    print(f"Colonne trovate per base '{question_base}': {columns_for_question}")
    
    if len(columns_for_question) == 1:
        print("-> Dovrebbe essere classificata come 'simple'")
    else:
        print("-> Dovrebbe essere classificata come 'multiple_choice'")
        
print(f"\n=== VERIFICA NEI RISULTATI ATTUALI ===")
# Controlla se è nei DataFrame generati
if 'questions_summary_df' in locals():
    questions_1_2 = questions_summary_df[questions_summary_df['question_base'].str.startswith('1.2')]
    print(f"Domande 1.2 in questions_summary_df: {len(questions_1_2)}")
    if len(questions_1_2) > 0:
        print(questions_1_2[['question_base', 'question_type', 'columns_count']])
else:
    print("questions_summary_df non trovato")

=== DIAGNOSI PERCHÉ LA DOMANDA 1.2 NON È NELL'EXPORT ===
Colonne che iniziano con '1.2': ['1.2 Età:']

Valori unici in '1.2 Età:': ['più di 60']
Tipo di valori: <class 'str'>

Base questions per 1.2 trovate: {'1.2 Età'}
Colonne trovate per base '1.2 Età': ['1.2 Età:']
-> Dovrebbe essere classificata come 'simple'

=== VERIFICA NEI RISULTATI ATTUALI ===
Domande 1.2 in questions_summary_df: 1
   question_base question_type  columns_count
41       1.2 Età        simple              1


In [6]:
# === CONTROLLO COMPLETO NUMERAZIONE DOMANDE ===
print("=== ANALISI COMPLETA NUMERAZIONE DOMANDE ===")

# Estrai tutti i numeri di domanda dal dataset
import re
from collections import defaultdict

# Pattern per estrarre numeri di domanda (es: 1.1, 2.3, 3.15, etc.)
question_pattern = re.compile(r'^(\d+)\.(\d+)')
sections = defaultdict(list)

print("Domande trovate nel dataset:")
for col in sorted(df.columns):
    match = question_pattern.match(col)
    if match:
        section = int(match.group(1))
        question_num = int(match.group(2))
        sections[section].append(question_num)
        print(f"  - {section}.{question_num} ({col[:60]}{'...' if len(col) > 60 else ''})")

print(f"\n=== ANALISI PER SEZIONE ===")
all_missing = []
for section in sorted(sections.keys()):
    question_numbers = sorted(set(sections[section]))
    print(f"\nSezione {section}:")
    print(f"  Domande trovate: {question_numbers}")
    
    # Controlla se ci sono numeri saltati
    min_num = min(question_numbers)
    max_num = max(question_numbers)
    expected = list(range(min_num, max_num + 1))
    missing = [n for n in expected if n not in question_numbers]
    
    if missing:
        missing_questions = [f"{section}.{n}" for n in missing]
        all_missing.extend(missing_questions)
        print(f"  ❌ DOMANDE MANCANTI: {missing_questions}")
    else:
        print(f"  ✅ Numerazione continua da {section}.{min_num} a {section}.{max_num}")

print(f"\n=== RIEPILOGO FINALE ===")
if all_missing:
    print(f"❌ DOMANDE SALTATE NELLA NUMERAZIONE ({len(all_missing)}):")
    for missing_q in all_missing:
        print(f"  - {missing_q}")
else:
    print("✅ NESSUNA DOMANDA SALTATA: la numerazione è continua in tutte le sezioni")

print(f"\n=== STATISTICHE ===")
total_questions = sum(len(set(questions)) for questions in sections.values())
total_sections = len(sections)
print(f"- Sezioni totali: {total_sections} (da {min(sections.keys())} a {max(sections.keys())})")
print(f"- Domande totali: {total_questions}")
print(f"- Domande mancanti: {len(all_missing)}")

# Verifica specifica per le domande che erano problematiche prima
problematic_questions = ['1.2', '2.3', '3.2', '3.7', '7.1']
print(f"\n=== VERIFICA DOMANDE SPECIFICHE ===")
for q in problematic_questions:
    section, num = q.split('.')
    if int(section) in sections and int(num) in sections[int(section)]:
        print(f"✅ {q}: PRESENTE")
    else:
        print(f"❌ {q}: MANCANTE")

=== ANALISI COMPLETA NUMERAZIONE DOMANDE ===
Domande trovate nel dataset:
  - 1.1 (1.1 Ruolo:)
  - 1.1 (1.1 Ruolo: [Altro])
  - 1.2 (1.2 Età:)
  - 1.3 (1.3 Genere:)
  - 1.4 (1.4 Titolo di studio (indichi tutti i titoli posseduti): [co...)
  - 1.4 (1.4 Titolo di studio (indichi tutti i titoli posseduti): [co...)
  - 1.4 (1.4 Titolo di studio (indichi tutti i titoli posseduti): [do...)
  - 1.4 (1.4 Titolo di studio (indichi tutti i titoli posseduti): [la...)
  - 1.4 (1.4 Titolo di studio (indichi tutti i titoli posseduti): [ma...)
  - 1.5 (1.5 Anzianità di servizio non di ruolo nella scuola:)
  - 1.6 (1.6 Anzianità di servizio di ruolo nella scuola:)
  - 1.7 (1.7 Attualmente svolge altri incarichi (vicepreside, funzion...)
  - 1.7 (1.7.b Specificare:)
  - 1.8 (1.8 Per quale classe di concorso ha conseguito l’abilitazion...)
  - 1.9 (1.9 Insegnamento in qualità di docente per le attività di so...)
  - 2.1 (2.1 Nel PTOF della sua scuola è prevista una sezione dedicat...)
  - 2.2 (2.2 Le at

In [15]:
# === RIEPILOGO SINTETICO DOMANDE MANCANTI ===
print("=== RIEPILOGO RAPIDO NUMERAZIONE DOMANDE ===")

# Estrai tutti i numeri di domanda
question_pattern = re.compile(r'^(\d+)\.(\d+)')
sections = defaultdict(list)

for col in df.columns:
    match = question_pattern.match(col)
    if match:
        section = int(match.group(1))
        question_num = int(match.group(2))
        sections[section].append(question_num)

# Trova lacune per ogni sezione
all_missing = []
print("Analisi per sezione:")
for section in sorted(sections.keys()):
    question_numbers = sorted(set(sections[section]))
    min_num = min(question_numbers)
    max_num = max(question_numbers)
    expected = list(range(min_num, max_num + 1))
    missing = [n for n in expected if n not in question_numbers]
    
    if missing:
        missing_questions = [f"{section}.{n}" for n in missing]
        all_missing.extend(missing_questions)
        print(f"  Sezione {section}: MANCANTI {missing_questions} (range {section}.{min_num}-{section}.{max_num})")
    else:
        print(f"  Sezione {section}: ✅ OK (range {section}.{min_num}-{section}.{max_num})")

print(f"\n=== RISULTATO FINALE ===")
if all_missing:
    print(f"❌ TROVATE {len(all_missing)} DOMANDE SALTATE:")
    for missing_q in sorted(all_missing):
        print(f"  - {missing_q}")
else:
    print("✅ NESSUNA DOMANDA SALTATA: numerazione continua")

# Test delle domande specificamente problematiche
problematic_questions = ['1.2', '2.3', '3.2', '3.7', '7.1']
print(f"\nVerifica domande precedentemente mancanti:")
for q in problematic_questions:
    section, num = q.split('.')
    if int(section) in sections and int(num) in sections[int(section)]:
        print(f"  ✅ {q}: ORA PRESENTE")
    else:
        print(f"  ❌ {q}: ANCORA MANCANTE")
        
print(f"\nStatistiche: {sum(len(set(q)) for q in sections.values())} domande in {len(sections)} sezioni")

=== RIEPILOGO RAPIDO NUMERAZIONE DOMANDE ===
Analisi per sezione:
  Sezione 1: ✅ OK (range 1.1-1.9)
  Sezione 2: ✅ OK (range 2.1-2.8)
  Sezione 3: ✅ OK (range 3.1-3.16)
  Sezione 4: ✅ OK (range 4.1-4.7)
  Sezione 5: ✅ OK (range 5.1-5.2)
  Sezione 6: ✅ OK (range 6.1-6.4)
  Sezione 7: ✅ OK (range 7.1-7.1)

=== RISULTATO FINALE ===
✅ NESSUNA DOMANDA SALTATA: numerazione continua

Verifica domande precedentemente mancanti:
  ✅ 1.2: ORA PRESENTE
  ✅ 2.3: ORA PRESENTE
  ✅ 3.2: ORA PRESENTE
  ✅ 3.7: ORA PRESENTE
  ✅ 7.1: ORA PRESENTE

Statistiche: 47 domande in 7 sezioni


In [19]:
# Verifica finale della correzione
questions_final = pd.read_csv('dataset_domande_con_opzioni.csv')
ruolo_final = questions_final[questions_final['question_base'] == '1.1 Ruolo']

print("=== VERIFICA FINALE ===")
print("Domanda: 1.1 Ruolo")
print(f"Tipo: {ruolo_final.iloc[0]['question_type']}")
print(f"Opzioni: {ruolo_final.iloc[0]['all_options']}")
print(f"Numero opzioni indicato: {ruolo_final.iloc[0]['options_count']}")
print(f"Numero opzioni reale: {len(ruolo_final.iloc[0]['all_options'].split('; '))}")
print(f"Colonne: {ruolo_final.iloc[0]['columns_count']}")

# Lista delle opzioni
options_list = ruolo_final.iloc[0]['all_options'].split('; ')
print("\nOpzioni nel dettaglio:")
for i, opt in enumerate(options_list, 1):
    print(f"  {i}. {opt}")
    
print(f"\n✓ Correzione completata! Ora la domanda '1.1 Ruolo' mostra correttamente {len(options_list)} opzioni.")

=== VERIFICA FINALE ===
Domanda: 1.1 Ruolo
Tipo: multiple_choice
Opzioni: docente; docente referente per l'orientamento e l'accoglienza; Altro: A022
Numero opzioni indicato: 3
Numero opzioni reale: 3
Colonne: 2

Opzioni nel dettaglio:
  1. docente
  2. docente referente per l'orientamento e l'accoglienza
  3. Altro: A022

✓ Correzione completata! Ora la domanda '1.1 Ruolo' mostra correttamente 3 opzioni.


In [41]:
# Esportazione dei file in formato Markdown con nome strutturato
import os
from datetime import datetime

# Configurazione per l'esportazione
PROJECT_NAME = "analisi-lime-survey-orientamento-CPIA"  # Nome della ricerca
EXPORT_DATE = datetime.now().strftime("%Y%m%d_%H%M%S")  # Data e ora corrente
EXPORT_FOLDER = f"export_md_{EXPORT_DATE}"

# Crea la cartella di esportazione
os.makedirs(EXPORT_FOLDER, exist_ok=True)

print(f"=== ESPORTAZIONE IN FORMATO MARKDOWN ===")
print(f"Progetto: {PROJECT_NAME}")
print(f"Data esportazione: {EXPORT_DATE}")
print(f"Cartella: {EXPORT_FOLDER}")

# Lista dei file da esportare con le loro descrizioni
files_to_export = [
    {
        'source': 'dataset_domande_con_opzioni.csv',
        'name': f"{PROJECT_NAME}_riassunto_domande_{EXPORT_DATE}.md",
        'title': 'Riassunto Domande con Opzioni di Risposta',
        'description': 'Questo file contiene un riassunto di tutte le domande selezionate dal questionario con le relative opzioni di risposta.'
    },
    {
        'source': 'dataset_domande_dettaglio.csv', 
        'name': f"{PROJECT_NAME}_dettaglio_colonne_{EXPORT_DATE}.md",
        'title': 'Dettaglio Completo delle Colonne',
        'description': 'Questo file contiene il dettaglio completo di tutte le colonne selezionate dal dataset.'
    },
    {
        'source': 'questions_with_options.csv',
        'name': f"{PROJECT_NAME}_mapping_completo_{EXPORT_DATE}.md", 
        'title': 'Mapping Completo Domande-Opzioni',
        'description': 'Questo file contiene il mapping completo di tutte le domande del questionario con le loro opzioni.'
    },
    {
        'source': 'header_analysis.csv',
        'name': f"{PROJECT_NAME}_analisi_intestazioni_{EXPORT_DATE}.md",
        'title': 'Analisi delle Intestazioni del Dataset',
        'description': 'Questo file contiene l\'analisi statistica di tutte le colonne del dataset originale.'
    }
]

def csv_to_markdown_table(csv_file):
    """Converte un file CSV in una tabella Markdown"""
    try:
        df = pd.read_csv(csv_file)
        
        # Crea l'header della tabella
        headers = '| ' + ' | '.join(df.columns) + ' |'
        separator = '|' + '|'.join([' --- ' for _ in df.columns]) + '|'
        
        # Crea le righe della tabella
        rows = []
        for _, row in df.head(50).iterrows():  # Limita a 50 righe per non rendere il file troppo grande
            row_str = '| ' + ' | '.join([str(val).replace('|', '\\|').replace('\n', ' ') for val in row]) + ' |'
            rows.append(row_str)
        
        # Se ci sono più di 50 righe, aggiungi una nota
        footer = ""
        if len(df) > 50:
            footer = f"\n\n*Nota: Mostrate solo le prime 50 righe di {len(df)} totali.*"
        
        return headers + '\n' + separator + '\n' + '\n'.join(rows) + footer
    except Exception as e:
        return f"Errore nella conversione: {e}"

# Esporta ogni file
exported_files = []
for file_info in files_to_export:
    if os.path.exists(file_info['source']):
        output_path = os.path.join(EXPORT_FOLDER, file_info['name'])
        
        # Crea il contenuto Markdown
        markdown_content = f"""# {file_info['title']}

**Progetto:** {PROJECT_NAME}  
**Data esportazione:** {datetime.now().strftime("%d/%m/%Y alle %H:%M:%S")}  
**File sorgente:** `{file_info['source']}`

## Descrizione

{file_info['description']}

## Dati

{csv_to_markdown_table(file_info['source'])}

---
*Generato automaticamente dal pipeline di analisi del questionario LIME Survey*
"""
        
        # Salva il file Markdown
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        
        exported_files.append(output_path)
        print(f"✓ Esportato: {file_info['name']}")
    else:
        print(f"✗ File non trovato: {file_info['source']}")

print(f"\n=== RISULTATO ESPORTAZIONE ===")
print(f"File esportati: {len(exported_files)}")
print(f"Cartella: {EXPORT_FOLDER}")
print("\nFile creati:")
for file_path in exported_files:
    file_size = os.path.getsize(file_path)
    print(f"  - {os.path.basename(file_path)} ({file_size} bytes)")

# Crea anche un file indice
index_content = f"""# Indice Esportazione - {PROJECT_NAME}

**Data esportazione:** {datetime.now().strftime("%d/%m/%Y alle %H:%M:%S")}

## File esportati

"""

for i, file_info in enumerate(files_to_export, 1):
    if os.path.exists(os.path.join(EXPORT_FOLDER, file_info['name'])):
        index_content += f"{i}. **[{file_info['title']}](./{file_info['name']})** - {file_info['description']}\n\n"

index_content += f"""
## Informazioni tecniche

- **Progetto:** {PROJECT_NAME}
- **Dataset originale:** merged_results.xlsx
- **Colonne selezionate:** {len(keep)} su {len(df.columns)} totali
- **Righe nel dataset:** {len(df)}

---
*Generato automaticamente dal pipeline di analisi del questionario LIME Survey*
"""

index_path = os.path.join(EXPORT_FOLDER, f"{PROJECT_NAME}_indice_{EXPORT_DATE}.md")
with open(index_path, 'w', encoding='utf-8') as f:
    f.write(index_content)

print(f"\n✓ Creato file indice: {os.path.basename(index_path)}")
print(f"\n🎯 Esportazione completata! Tutti i file sono nella cartella '{EXPORT_FOLDER}'")

=== ESPORTAZIONE IN FORMATO MARKDOWN ===
Progetto: analisi-lime-survey-orientamento-CPIA
Data esportazione: 20250924_114858
Cartella: export_md_20250924_114858
✓ Esportato: analisi-lime-survey-orientamento-CPIA_riassunto_domande_20250924_114858.md
✓ Esportato: analisi-lime-survey-orientamento-CPIA_dettaglio_colonne_20250924_114858.md
✓ Esportato: analisi-lime-survey-orientamento-CPIA_mapping_completo_20250924_114858.md
✓ Esportato: analisi-lime-survey-orientamento-CPIA_analisi_intestazioni_20250924_114858.md

=== RISULTATO ESPORTAZIONE ===
File esportati: 4
Cartella: export_md_20250924_114858

File creati:
  - analisi-lime-survey-orientamento-CPIA_riassunto_domande_20250924_114858.md (80945 bytes)
  - analisi-lime-survey-orientamento-CPIA_dettaglio_colonne_20250924_114858.md (21875 bytes)
  - analisi-lime-survey-orientamento-CPIA_mapping_completo_20250924_114858.md (21883 bytes)
  - analisi-lime-survey-orientamento-CPIA_analisi_intestazioni_20250924_114858.md (13760 bytes)

✓ Creato fi

In [57]:
# 4) Analisi domande: raggruppamento e rilevazione Likert (senza grafici)
import re, pandas as pd, unicodedata

# 4.1 Carica dataset (se non esiste già in memoria)
try:
    _ = data  # verifica esistenza
except NameError:
    try:
        data = pd.read_csv('dataset_domande.csv')
    except FileNotFoundError:
        data = pd.read_excel('merged_results.xlsx')

# 4.2 Utility testo

def strip_accents(s: str) -> str:
    if s is None:
        return ''
    s = str(s)
    nfkd = unicodedata.normalize('NFKD', s)
    return ''.join(ch for ch in nfkd if not unicodedata.combining(ch))

def norm_txt(s: str) -> str:
    s = strip_accents(s).lower().strip()
    s = re.sub(r'\s+', ' ', s)
    return s

# 4.3 Raggruppa colonne per prefisso numerico (es. '2.3')
num_pat = re.compile(r'^(\d+\.\d+)(?:[\s\S]*)$')
question_groups = {}
for col in data.columns:
    m = num_pat.match(col)
    if m:
        key = m.group(1)
        question_groups.setdefault(key, []).append(col)

# 4.4 Etichetta leggibile domanda (numero — testo)
clean_q_text_pat_prefix = re.compile(r'^\d+\.\d+\s*')
clean_q_text_pat_brackets = re.compile(r'\s*\[[^\]]+\]\s*$')

def clean_question_text(col: str) -> str:
    s = str(col)
    s = clean_q_text_pat_prefix.sub('', s).strip()
    s = clean_q_text_pat_brackets.sub('', s).strip()
    s = s.rstrip(':').strip()
    return s

group_labels = {}
for key, cols in question_groups.items():
    texts = [clean_question_text(c) for c in cols if c]
    group_labels[key] = max(set(texts), key=lambda t: (texts.count(t), len(t))) if texts else key

# 4.5 Definizione famiglie Likert e token mapping
LIKERT_FAMILIES = {
    'intensita': {
        'order': ['Per nulla', 'Poco', 'Abbastanza', 'Molto', 'Moltissimo'],
        'tokens': {
            'per nulla': 'Per nulla', 'per niente': 'Per nulla', 'niente affatto': 'Per nulla',
            'poco': 'Poco',
            'abbastanza': 'Abbastanza', 'sufficientemente': 'Abbastanza', 'mediamente': 'Abbastanza',
            'molto': 'Molto', 'tanto': 'Molto',
            'moltissimo': 'Moltissimo', 'estremamente': 'Moltissimo'
        }
    },
    'accordo': {
        'order': ["Per nulla d'accordo", "In disaccordo", 'Neutrale', "D'accordo", "Molto d'accordo"],
        'tokens': {
            "per nulla d'accordo": "Per nulla d'accordo", 'fortemente in disaccordo': "Per nulla d'accordo",
            'in disaccordo': 'In disaccordo',
            'neutrale': 'Neutrale', "ne d'accordo ne in disaccordo": 'Neutrale', 'indifferente': 'Neutrale',
            "d'accordo": "D'accordo",
            "molto d'accordo": "Molto d'accordo", "completamente d'accordo": "Molto d'accordo"
        }
    },
    'frequenza': {
        'order': ['Mai', 'Raramente', 'A volte', 'Spesso', 'Sempre'],
        'tokens': {
            'mai': 'Mai',
            'raramente': 'Raramente', 'poco spesso': 'Raramente',
            'a volte': 'A volte', 'talvolta': 'A volte', 'occasionalmente': 'A volte',
            'spesso': 'Spesso', 'frequentemente': 'Spesso',
            'sempre': 'Sempre', 'quasi sempre': 'Sempre'
        }
    },
    'qualita': {
        'order': ['Insufficiente', 'Sufficiente', 'Buona', 'Ottima'],
        'tokens': {
            'insufficiente': 'Insufficiente', 'scarsa': 'Insufficiente', 'pessima': 'Insufficiente',
            'sufficiente': 'Sufficiente', 'discreta': 'Sufficiente',
            'buona': 'Buona',
            'ottima': 'Ottima', 'eccellente': 'Ottima'
        }
    }
}

# 4.6 Helper per rilevazione Likert e mapping

def first_non_na(series: pd.Series):
    for v in series:
        if pd.notna(v) and str(v).strip() != '':
            return v
    return None

def guess_family_from_first_row(cols):
    fam_counts = {}
    for c in cols:
        v = first_non_na(data[c])
        if v is None:
            continue
        nv = norm_txt(str(v))
        matched_family = None
        for fam, cfg in LIKERT_FAMILIES.items():
            for token in cfg['tokens'].keys():
                if token in nv:
                    matched_family = fam
                    break
            if matched_family:
                break
        if matched_family:
            fam_counts[matched_family] = fam_counts.get(matched_family, 0) + 1
    return max(fam_counts, key=fam_counts.get) if fam_counts else None

def map_value_to_canonical(v: str, family: str):
    if pd.isna(v) or str(v).strip() == '':
        return None
    nv = norm_txt(str(v))
    for token, canon in LIKERT_FAMILIES[family]['tokens'].items():
        if token in nv:
            return canon
    return None

# 4.7 Rileva famiglia Likert per ogni gruppo
_group_families = {g: guess_family_from_first_row(cols) for g, cols in question_groups.items()}
likert_summary = pd.DataFrame([
    {
        'group': g,
        'label': group_labels.get(g, g),
        'family': _group_families[g] if _group_families[g] else 'non-likert',
        'n_cols': len(question_groups[g])
    }
    for g in sorted(question_groups.keys(), key=lambda k: (int(k.split('.')[0]), int(k.split('.')[1])))
])

# Mostra un estratto della sintesi
likert_summary.head(200)

,group,label,family,n_cols
0,1.1,Ruolo,non-likert,2
1,1.2,Età,non-likert,1
2,1.3,Genere,non-likert,1
3,1.4,Titolo di studio (indichi tutti i titoli posse...,non-likert,5
4,1.5,Anzianità di servizio non di ruolo nella scuola,non-likert,1
5,1.6,Anzianità di servizio di ruolo nella scuola,non-likert,1
6,1.7,Attualmente svolge altri incarichi (vicepresid...,non-likert,2
7,1.8,Per quale classe di concorso ha conseguito l’a...,non-likert,1
8,1.9,Insegnamento in qualità di docente per le atti...,frequenza,1
9,2.1,Nel PTOF della sua scuola è prevista una sezio...,non-likert,1


In [58]:
#  [Diagnostica] Ambiente e renderer Plotly
import sys, plotly
import plotly.io as pio
from packaging import version

# Rileva versione nbformat se presente e compatibile
try:
    import nbformat  # noqa: F401
    nbv = nbformat.__version__
    has_compatible_nbformat = version.parse(nbv) >= version.parse("4.2.0")
except Exception as e:
    nbv = f"missing ({e})"
    has_compatible_nbformat = False

# Funzione robusta per impostare un renderer sicuro in questo ambiente
def _configure_plotly_renderer():
    available = set(pio.renderers)  # renderers disponibili in questa sessione
    # Preferenze in ordine: plotly_mimetype (se nbformat>=4.2.0), vscode, notebook_connected, browser
    if has_compatible_nbformat and 'plotly_mimetype' in available:
        pio.renderers.default = 'plotly_mimetype'
        return
    if 'vscode' in available:
        pio.renderers.default = 'vscode'
        return
    if 'notebook_connected' in available:
        pio.renderers.default = 'notebook_connected'
        return
    # Fallback universale: apre nel browser di sistema
    pio.renderers.default = 'browser'

_configure_plotly_renderer()

print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print("Plotly:", plotly.__version__)
print("nbformat:", nbv)
print("nbformat compatible:", has_compatible_nbformat)
print("Renderer:", pio.renderers.default)

ModuleNotFoundError: No module named 'plotly'

In [58]:
# 🎯 ANALISI COMPLETA SENZA WIDGET
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import numpy as np
import textwrap
try:
    import scipy.stats as stats
except ImportError:
    print("⚠️  scipy non disponibile - grafici gaussiani limitati")
    stats = None

def wrap_title(title, max_chars=140):
    """
    Fa andare a capo i titoli lunghi per i grafici
    """
    if len(title) <= max_chars:
        return title
    
    # Usa textwrap per dividere il testo in righe
    wrapped = textwrap.fill(title, width=max_chars)
    return wrapped.replace('\n', '<br>')  # Plotly usa <br> per andare a capo

def analyze_question_group(group_key='2.4', chart_type='bar', show_percentages=True, include_na=False):
    """
    Analizza un gruppo di domande con statistiche complete e tipi di grafico
    
    Parametri:
    - group_key: chiave del gruppo (es. '2.4')
    - chart_type: 'bar', 'bar_h', 'pie', 'likert_bar', 'histogram', 'gaussian'
    - show_percentages: True per %, False per numeri assoluti (quando applicabile)
    - include_na: include valori NA nell'analisi
    """
    print(f"🎯 ANALISI DOMANDA: {group_key}")
    print(f"📝 Descrizione: {group_labels.get(group_key, group_key)}")
    print(f"📊 Tipo grafico: {chart_type}")
    print(f"📈 Valori: {'Percentuali' if show_percentages else 'Numeri assoluti'}")
    print(f"🔢 Includi NA: {include_na}")
    
    # Spiegazione del tipo di grafico
    chart_explanations = {
        'bar': "📊 Barre verticali - Ideale per confrontare categorie, facile da leggere",
        'bar_h': "📊 Barre orizzontali - Migliore quando le etichette sono lunghe",
        'pie': "🥧 Grafico a torta - Mostra proporzioni del totale, max 5-6 categorie",
        'likert_bar': "📈 Barre Likert - Ordinate secondo scala (Per nulla → Molto)",
        'histogram': "📉 Istogramma - Mostra distribuzione di valori numerici",
        'gaussian': "🔔 Curva gaussiana - Istogramma + curva normale sovrapposta"
    }
    
    if chart_type in chart_explanations:
        print(f"💡 {chart_explanations[chart_type]}")
    print()
    
    if group_key not in question_groups:
        print(f"❌ Gruppo {group_key} non trovato")
        return
    
    cols = question_groups[group_key]
    print(f"🔢 Numero di sotto-domande: {len(cols)}")
    print("=" * 80)
    
    # Per ogni colonna del gruppo, crea un grafico
    for i, col in enumerate(cols, 1):
        print(f"📊 Sotto-domanda {i}:")
        print(f"    {col}")
        
        # Prendi i dati della colonna
        series = data[col]
        original_count = len(series)
        
        if not include_na:
            series = series.dropna()
        
        if len(series) == 0:
            print("   ⚠️  Nessun dato disponibile")
            continue
            
        # Conta i valori
        counts = Counter(series)
        total = sum(counts.values())
        
        # Statistiche descrittive
        print(f"   📈 Statistiche:")
        print(f"      • Risposte totali: {total}")
        if not include_na and original_count > total:
            print(f"      • Valori mancanti: {original_count - total}")
        
        # Calcola statistiche numeriche se applicabile
        try:
            # Prova a mappare i valori Likert a numeri per le statistiche
            likert_family = _group_families.get(group_key)
            if likert_family and likert_family in LIKERT_FAMILIES:
                order = LIKERT_FAMILIES[likert_family]['order']
                # Mappa i valori a numeri (1, 2, 3, ...)
                numeric_values = []
                for val in series:
                    try:
                        idx = order.index(val)
                        numeric_values.append(idx + 1)
                    except ValueError:
                        continue
                
                if numeric_values:
                    mean_val = np.mean(numeric_values)
                    median_val = np.median(numeric_values)
                    std_val = np.std(numeric_values, ddof=1) if len(numeric_values) > 1 else 0
                    
                    print(f"      • Media: {mean_val:.2f}")
                    print(f"      • Mediana: {median_val:.1f}")
                    print(f"      • Deviazione standard: {std_val:.2f}")
        except Exception:
            pass
        
        # Distribuzione dei valori
        print(f"   📊 Distribuzione:")
        for value, count in counts.most_common():
            pct = round(100 * count / total, 1) if total > 0 else 0
            print(f"      • {value}: {count} ({pct}%)")
        
        # Crea grafico in base al tipo richiesto
        if len(counts) > 0:
            labels = list(counts.keys())
            values = list(counts.values())
            
            # Palette di colori vivaci per tutti i grafici
            colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', 
                     '#FF9FF3', '#54A0FF', '#5F27CD', '#00D2D3', '#FF9F43',
                     '#6C5CE7', '#A29BFE', '#FD79A8', '#E17055', '#00B894']
            
            # Titolo con a capo automatico
            wrapped_title = wrap_title(col)
            
            # Calcola valori per i grafici - sempre mostra entrambi
            percentages = [round(100 * v / total, 1) for v in values]
            # Testo che mostra sia numero assoluto che percentuale
            text_labels = [f"{v} ({p}%)" for v, p in zip(values, percentages)]
            
            # Display values per l'asse Y
            display_values = percentages if show_percentages else values
            y_label = 'Percentuale (%)' if show_percentages else 'Conteggio'
            
            if chart_type == 'bar':
                # Grafico a barre verticali
                fig = px.bar(
                    x=labels, 
                    y=display_values,
                    title=wrapped_title,
                    labels={'x': 'Risposta', 'y': y_label},
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            elif chart_type == 'bar_h':
                # Grafico a barre orizzontali
                fig = px.bar(
                    x=display_values,
                    y=labels,
                    title=wrapped_title,
                    labels={'x': y_label, 'y': 'Risposta'},
                    orientation='h',
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            elif chart_type == 'pie':
                # Grafico a torta con etichette personalizzate
                fig = px.pie(
                    names=labels,
                    values=values,  # Sempre valori assoluti per le torte
                    title=wrapped_title,
                    color_discrete_sequence=colors
                )
                # Aggiunge etichette personalizzate alla torta
                fig.update_traces(textinfo='label+value+percent', 
                                texttemplate='%{label}<br>%{value} (%{percent})')
            elif chart_type == 'likert_bar':
                # Grafico Likert ordinato
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    ordered_labels = [l for l in order if l in labels]
                    ordered_values = [counts[l] for l in ordered_labels]
                    ordered_display = [round(100 * v / total, 1) for v in ordered_values] if show_percentages else ordered_values
                    ordered_text = [f"{v} ({round(100 * v / total, 1)}%)" for v in ordered_values]
                    
                    fig = px.bar(
                        x=ordered_labels, 
                        y=ordered_display,
                        title=wrapped_title,
                        labels={'x': 'Risposta', 'y': y_label},
                        text=ordered_text,
                        color=ordered_labels,
                        color_discrete_sequence=colors
                    )
                else:
                    # Fallback a barre normali se non è Likert
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               labels={'x': 'Risposta', 'y': y_label}, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            elif chart_type == 'histogram':
                # Istogramma (per dati numerici Likert)
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    numeric_data = []
                    for val in series:
                        try:
                            idx = order.index(val)
                            numeric_data.extend([idx + 1] * 1)  # Mappa a numeri 1,2,3...
                        except ValueError:
                            continue
                    
                    if numeric_data:
                        fig = px.histogram(
                            x=numeric_data,
                            title=wrapped_title,
                            labels={'x': 'Valore Likert (1=Per nulla, 5=Moltissimo)', 'y': 'Frequenza'},
                            nbins=len(order),
                            color_discrete_sequence=[colors[0]]
                        )
                    else:
                        fig = px.bar(x=labels, y=display_values, title=wrapped_title,
                                   text=text_labels,
                                   color=labels, color_discrete_sequence=colors)
                else:
                    # Per dati non-Likert, usa grafico a barre normale
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            elif chart_type == 'gaussian':
                # Curva gaussiana sovrapposta
                likert_family = _group_families.get(group_key)
                if likert_family and likert_family in LIKERT_FAMILIES:
                    order = LIKERT_FAMILIES[likert_family]['order']
                    numeric_data = []
                    for val in series:
                        try:
                            idx = order.index(val)
                            numeric_data.append(idx + 1)
                        except ValueError:
                            continue
                    
                    if numeric_data and len(numeric_data) > 1:
                        import scipy.stats as stats
                        
                        # Crea istogramma
                        fig = px.histogram(
                            x=numeric_data,
                            title=wrapped_title,
                            labels={'x': 'Valore Likert', 'y': 'Frequenza'},
                            nbins=len(order),
                            opacity=0.7,
                            color_discrete_sequence=[colors[0]]
                        )
                        
                        # Aggiunge curva gaussiana
                        x_range = np.linspace(min(numeric_data), max(numeric_data), 100)
                        mean_val = np.mean(numeric_data)
                        std_val = np.std(numeric_data)
                        gaussian_curve = stats.norm.pdf(x_range, mean_val, std_val)
                        # Scala la curva per matchare l'istogramma
                        gaussian_curve = gaussian_curve * len(numeric_data) * (max(numeric_data) - min(numeric_data)) / len(order)
                        
                        fig.add_scatter(
                            x=x_range, 
                            y=gaussian_curve,
                            mode='lines',
                            name='Curva Normale',
                            line=dict(color='red', width=3)
                        )
                    else:
                        fig = px.bar(x=labels, y=display_values, title=wrapped_title,
                                   text=text_labels,
                                   color=labels, color_discrete_sequence=colors)
                else:
                    fig = px.bar(x=labels, y=display_values, title=wrapped_title, 
                               text=text_labels,
                               color=labels, color_discrete_sequence=colors)
            else:
                # Default: barre verticali
                fig = px.bar(
                    x=labels, 
                    y=display_values,
                    title=wrapped_title,
                    labels={'x': 'Risposta', 'y': y_label},
                    text=text_labels,
                    color=labels,
                    color_discrete_sequence=colors
                )
            
            # Styling avanzato con margine superiore dinamico per titoli lunghi
            title_lines = wrapped_title.count('<br>') + 1
            top_margin = 100 + (title_lines - 1) * 30  # Margine superiore più ampio
            
            # Calcola margine sinistro dinamico per etichette lunghe
            max_label_length = max(len(str(label)) for label in labels) if labels else 0
            left_margin = max(80, min(200, 80 + max_label_length * 3))  # Margine sinistro adattivo
            
            # Altezza adattiva basata sul numero di elementi
            adaptive_height = max(500, 400 + len(labels) * 20)  # Altezza minima 500px
            
            fig.update_layout(
                xaxis_tickangle=-45,
                height=adaptive_height,
                margin={
                    'l': left_margin, 
                    'r': 80,  # Margine destro più ampio
                    't': top_margin, 
                    'b': 150  # Margine inferiore più ampio per etichette inclinate
                },
                plot_bgcolor='rgba(240,240,245,0.8)',
                paper_bgcolor='white',
                font=dict(size=12),
                title_font=dict(size=14, color='#2c3e50'),
                showlegend=False,  # Nasconde leggenda per tutti i grafici tranne torte
                xaxis=dict(
                    title_font=dict(size=12, color='#34495e'),
                    tickfont=dict(size=10, color='#2c3e50'),
                    gridcolor='rgba(200,200,200,0.3)',
                    automargin=True  # Margine automatico per evitare tagli
                ) if chart_type != 'pie' else {},
                yaxis=dict(
                    title_font=dict(size=12, color='#34495e'),
                    tickfont=dict(size=10, color='#2c3e50'),
                    gridcolor='rgba(200,200,200,0.3)',
                    automargin=True  # Margine automatico per evitare tagli
                ) if chart_type != 'pie' else {}
            )
            
            # Per grafici a torta, mostra la leggenda se ci sono molte categorie
            if chart_type == 'pie':
                fig.update_layout(showlegend=(len(labels) > 4))
            
            # Aggiunge dettagli alle barre (eccetto torta)
            if chart_type != 'pie':
                fig.update_traces(
                    marker_line_color='white',
                    marker_line_width=2,
                    opacity=0.85,
                    texttemplate='%{text}',  # Usa il testo personalizzato
                    textposition='outside',
                    textfont_size=11,
                    textfont_color='#2c3e50',
                    hovertemplate='<b>%{x}</b><br>%{text}<br><extra></extra>'  # Tooltip pulito
                )
            else:
                # Per grafici a torta, personalizza il tooltip
                fig.update_traces(
                    hovertemplate='<b>%{label}</b><br>%{value} (%{percent})<extra></extra>'
                )
            
            fig.show()
        
        print("-" * 50)
    
    print("=" * 80)

# Funzioni di supporto per diversi tipi di analisi
def analyze_bar(group_key, percentages=True, include_na=False):
    """Analisi con grafici a barre verticali"""
    return analyze_question_group(group_key, 'bar', percentages, include_na)

def analyze_bar_h(group_key, percentages=True, include_na=False):
    """Analisi con grafici a barre orizzontali"""
    return analyze_question_group(group_key, 'bar_h', percentages, include_na)

def analyze_pie(group_key, include_na=False):
    """Analisi con grafici a torta"""
    return analyze_question_group(group_key, 'pie', True, include_na)

def analyze_likert(group_key, percentages=True, include_na=False):
    """Analisi con grafici Likert ordinati"""
    return analyze_question_group(group_key, 'likert_bar', percentages, include_na)

def analyze_histogram(group_key, include_na=False):
    """Analisi con istogramma (per dati Likert numerici)"""
    return analyze_question_group(group_key, 'histogram', True, include_na)

def analyze_gaussian(group_key, include_na=False):
    """Analisi con curva gaussiana sovrapposta"""
    return analyze_question_group(group_key, 'gaussian', True, include_na)

# 🎯 Funzioni di analisi pronte!
print("🎯 Funzioni di analisi pronte!")
print()
print("📊 Uso completo:")
print("   analyze_question_group('2.4', 'bar', True, False)")
print("   → ('domanda', 'tipo_grafico', percentuali?, includi_NA?)")
print()
print("📈 Tipi di grafico:")
print("   'bar'         → Barre verticali")
print("   'bar_h'       → Barre orizzontali") 
print("   'pie'         → Grafici a torta")
print("   'likert_bar'  → Barre Likert ordinate")
print("   'histogram'   → Istogramma") 
print("   'gaussian'    → Curva gaussiana")
print()
print("🚀 Funzioni rapide:")
print("   analyze_bar('2.4')        # Barre verticali %")
print("   analyze_bar_h('2.4')      # Barre orizzontali %") 
print("   analyze_pie('2.4')        # Torte")
print("   analyze_likert('2.4')     # Likert %")
print("   analyze_histogram('2.4')  # Istogramma")
print("   analyze_gaussian('2.4')   # Curva normale")
print()
print("💡 Per numeri assoluti: analyze_bar('2.4', percentages=False)")
print()
print(f"📋 Gruppi disponibili: {sorted(question_groups.keys())[:10]}...")

🎯 Funzioni di analisi pronte!

📊 Uso completo:
   analyze_question_group('2.4', 'bar', True, False)
   → ('domanda', 'tipo_grafico', percentuali?, includi_NA?)

📈 Tipi di grafico:
   'bar'         → Barre verticali
   'bar_h'       → Barre orizzontali
   'pie'         → Grafici a torta
   'likert_bar'  → Barre Likert ordinate
   'histogram'   → Istogramma
   'gaussian'    → Curva gaussiana

🚀 Funzioni rapide:
   analyze_bar('2.4')        # Barre verticali %
   analyze_bar_h('2.4')      # Barre orizzontali %
   analyze_pie('2.4')        # Torte
   analyze_likert('2.4')     # Likert %
   analyze_histogram('2.4')  # Istogramma
   analyze_gaussian('2.4')   # Curva normale

💡 Per numeri assoluti: analyze_bar('2.4', percentages=False)

📋 Gruppi disponibili: ['1.1', '1.10', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9']...


In [59]:
# 🧪 TEST: Analisi completa con statistiche
# Cambia questi parametri per testare diversi tipi di analisi:

GROUP = '1.1'           # Quale domanda analizzare
CHART_TYPE = 'bar'      # Tipo grafico (vedi spiegazioni sotto)
SHOW_PERCENTAGES = True # True = percentuali, False = numeri assoluti
INCLUDE_NA = False      # Includere valori mancanti

# 📊 TIPI DI GRAFICO DISPONIBILI:
# 'bar'         → Barre verticali (più leggibile)
# 'bar_h'       → Barre orizzontali (per etichette lunghe)  
# 'pie'         → Torta (buono per poche categorie)
# 'likert_bar'  → Barre Likert ordinate (Per nulla → Molto)
# 'histogram'   → Istogramma per distribuzioni numeriche
# 'gaussian'    → Curva gaussiana sovrapposta all'istogramma

# Esegui analisi
analyze_question_group(GROUP, CHART_TYPE, SHOW_PERCENTAGES, INCLUDE_NA)

🎯 ANALISI DOMANDA: 1.1
📝 Descrizione: Ruolo
📊 Tipo grafico: bar
📈 Valori: Percentuali
🔢 Includi NA: False
💡 📊 Barre verticali - Ideale per confrontare categorie, facile da leggere

🔢 Numero di sotto-domande: 2
📊 Sotto-domanda 1:
    1.1 Ruolo:
   📈 Statistiche:
      • Risposte totali: 169
   📊 Distribuzione:
      • docente orientatore: 120 (71.0%)
      • docente tutor: 24 (14.2%)
      • Altro: 14 (8.3%)
      • dirigente scolastico: 11 (6.5%)


--------------------------------------------------
📊 Sotto-domanda 2:
    1.1 Ruolo: [Altro]
   📈 Statistiche:
      • Risposte totali: 14
      • Valori mancanti: 155
   📊 Distribuzione:
      • Funzione strumentale orientamento: 1 (7.1%)
      • Referente Invalsi e RAV: 1 (7.1%)
      • Funzione strumentale Orientamento: 1 (7.1%)
      • gestore scuola: 1 (7.1%)
      • figura strumentale orientamento : 1 (7.1%)
      • Responsabile orientamento indirizzo turistico: 1 (7.1%)
      • Coordinatrice didattica e docente orientatore: 1 (7.1%)
      • Referente orientamento in uscita: 1 (7.1%)
      • Vice dirigente : 1 (7.1%)
      • Funzione strumentale per l’orientamento in ingresso: 1 (7.1%)
      • Docente : 1 (7.1%)
      • Referente per l'Orientamento in Uscita: 1 (7.1%)
      • Tutor ed orientatore in uscita: 1 (7.1%)
      • F.S. Referente per l'Orientamento: 1 (7.1%)


--------------------------------------------------


In [ ]:
# 🎨 ESEMPI DI TUTTI I TIPI DI GRAFICO
# Decommmenta quello che vuoi testare:

# analyze_question_group('2.4', 'bar', True, False)          # Barre verticali %
# analyze_question_group('2.4', 'bar', False, False)         # Barre verticali numeri assoluti
# analyze_question_group('2.4', 'bar_h', True, False)        # Barre orizzontali %  
# analyze_question_group('2.4', 'pie', True, False)          # Torta
# analyze_question_group('2.4', 'likert_bar', True, False)   # Likert ordinato %
# analyze_question_group('2.4', 'histogram', True, False)    # Istogramma 
# analyze_question_group('2.4', 'gaussian', True, False)     # Curva gaussiana

# 🚀 Oppure usa le funzioni rapide:
# analyze_gaussian('2.4')   # Prova questo per vedere la curva normale!

print("✅ Esempi pronti! Decommenta una riga per testare i diversi grafici")